In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sm

In [59]:
sm_Rax = sm.symbols("Rax")
sm_Ray = sm.symbols("Ray")
sm_Raz = sm.symbols("Raz")
sm_Ra = [sm_Rax, sm_Ray, sm_Raz]

sm_Rbx = sm.symbols("Rbx")
sm_Rby = sm.symbols("Rby")
sm_Rbz = sm.symbols("Rbz")
sm_Rb = [sm_Rbx, sm_Rby, sm_Rbz]

sm_mu0 = sm.symbols("mu0")
sm_a = sm.symbols("a")
sm_d0 = sm.symbols("d0")
sm_d7 = sm.symbols("d7")

class GriegorievDipoleFunction:
    def __init__(self, mu0, a, d0, d7):

        self.parameters = [mu0, a, d0, d7]

        sm_d = ((sm_Rax - sm_Rbx)**2 + (sm_Ray - sm_Rby)**2 \
             + (sm_Raz - sm_Rbz)**2)**(1/2)
        
        self.dipole_exp = [
            ((sm_Ra[i] - sm_Rb[i])/d) * sm_mu0 * sm.exp(-sm_a*(sm_d-sm_d0)) # - d7/(d**7)
            for i in range(3)
        ]
        
        self.dipole_function = [
            sm.lambdify(
            [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
             sm_mu0, sm_a, sm_d0, sm_d7], 
                dipole_exp
        ) for dipole_exp in self.dipole_exp]

        self.generate_Jacobi()

    def generate_Jacobi(self):
        self.J_exp = []
        self.J_func = []

        for Ra in sm_Ra:
            J_sub_func = []
            for dipole_f in self.dipole_exp:
                d_mu = sm.diff(dipole_f, Ra)
                self.J_exp.append(d_mu)
                J_sub_func.append(sm.lambdify(
                    [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
                     sm_mu0, sm_a, sm_d0, sm_d7], d_mu
                ))
            self.J_func.append(J_sub_func)

    def generate_Hessian(self):
        self.H_exp = []
        self.H_func = []

    def __call__(self, ra, rb):
        ra = list(ra)
        rb = list(rb)

        args = ra + rb + self.parameters

        result = [dipole_f(*args) for dipole_f in self.dipole_function]
        
        return result

    def gradient(self,ra,rb):
        args = ra + rb + self.parameters
        result = np.zeros((3,3))
        for i, dmu_dRa in enumerate(self.J_func):
            for j, dmu_i_dRa in enumerate(dmu_dRa):
                result[i,j] = dmu_i_dRa(*args)

        return result

dipole_function = GriegorievDipoleFunction(*[1, 1.5121, 7.10, 300])

dipole_function.gradient([0,0,0],[3,4,1])


array([[ -8.1440906 , -16.24755337,  -4.06188834],
       [-16.24755337, -17.62183007,  -5.41585112],
       [ -4.06188834,  -5.41585112,   2.68761164]])

In [56]:
sm.diff(dipole_function.dipole_exp[0],sm_Ray)

-a*mu0*(Rax - Rbx)*(1.0*Ray - 1.0*Rby)*exp(-a*(-d0 + ((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**0.5))/((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**1.0 + mu0*(Rax - Rbx)*(-1.0*Ray + 1.0*Rby)*exp(-a*(-d0 + ((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**0.5))/((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**1.5

In [57]:
sm.diff(dipole_function.dipole_exp[1],sm_Rax)

-a*mu0*(1.0*Rax - 1.0*Rbx)*(Ray - Rby)*exp(-a*(-d0 + ((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**0.5))/((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**1.0 + mu0*(-1.0*Rax + 1.0*Rbx)*(Ray - Rby)*exp(-a*(-d0 + ((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**0.5))/((Rax - Rbx)**2 + (Ray - Rby)**2 + (Raz - Rbz)**2)**1.5